<a href="https://colab.research.google.com/github/jcs-lambda/DS-Unit-1-Sprint-2-Statistics/blob/master/module3/LS_DS_123_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function 

`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):` 

You should only truly need these two values in order to apply Bayes Theorem. In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000. 
 - What is the probability that a person is drunk after one positive breathalyzer test?
 - What is the probability that a person is drunk after two positive breathalyzer tests?
 - How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



### 1. Write a function

In [0]:
def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):
  complement_of_prior = 1 - prob_drunk_prior
  post_probability = prob_drunk_prior * true_positive_rate / ((prob_drunk_prior * true_positive_rate)+(complement_of_prior * false_positive_rate))
  return post_probability

In [11]:
#@title probability that a person is drunk after one test
prior_belief = 1 / 1000
false_rate = 0.08
true_rate = 1.0

prob_drunk_given_positive(prior_belief, false_rate, true_rate)

0.012357884330202669

In [12]:
#@title probability that person is drunk after 2 tests
new_belief = prob_drunk_given_positive(prior_belief, false_rate, true_rate)
newer_belief = prob_drunk_given_positive(new_belief, false_rate, true_rate)
newer_belief

0.13525210993291495

In [13]:
#@title how many tests needed to have a probability > 95%
test_count = 0
while prior_belief < 0.95:
  post_belief = prob_drunk_given_positive(prior_belief, false_rate, true_rate)
  prior_belief = post_belief
  test_count += 1

print(f'needed {test_count} tests to reach a probability of {prior_belief*100:.2f}%')

needed 4 tests to reach a probability of 96.07%


#### 1.b write a more generic function

In [0]:
def bayesion_iterator(prior_probability, false_positive_rate, true_positive_rate, desired_confidence_level=0.95):
  counter = 0
  results = {counter:prior_probability}
  while prior_probability < desired_confidence_level:
    complement_of_prior_probability = 1 - prior_probability
    prior_probability = prior_probability * true_positive_rate / ((prior_probability * true_positive_rate) + (complement_of_prior_probability * false_positive_rate))
    counter += 1
    results[counter] = prior_probability
  return results

In [15]:
#@title dui test probabilities until greater than 95%, given the above numbers

bayesion_iterator(1/1000, 0.08, 1.0)

{0: 0.001,
 1: 0.012357884330202669,
 2: 0.13525210993291495,
 3: 0.6615996951348605,
 4: 0.9606895076105054}

### 2. Explore scipy.stats.bayes_mvs

In [0]:
from scipy import stats

### 3. Summarize the difference between bayesion and frequentist statistics

- frequentist
 - relies on sample size for accuracy
 - probability is determined by analyzing a large set of distributions
- bayesian
 - works from a prior probability to calculate a new probability
 - requires known prior assumptions
 - revises and then uses those assumptions based on further testing


the more i try to read on this, the more confusing it becomes.
i think that means i don't understand it. at all.


## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!